<a href="https://colab.research.google.com/github/xpertmind/RecipeNLG/blob/main/setup_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Import the libraries needed and define some variables 


In [ ]:
!pip install pyTigerGraph
!pip install pandas
chunk_size = 5000
# Imports
import numpy as np 
import pandas as pd
import pyTigerGraph as tg
import re

  Created wheel for validators: filename=validators-0.19.0-py3-none-any.whl size=19553 sha256=dc33b447514358e7f102045e90c8599c818e0942ddbfd10f04bc10d764fc251d
  Stored in directory: /root/.cache/pip/wheels/fe/5d/69/ff53a908b9f14fb7730a58fdede0fac4cdc99ef3624ec76d05
Successfully built validators


Please put here your specific setup variables:

In [ ]:
tghost="https://bs-dis.i.tgcloud.io"
tguser="tigergraph" # default!
tgpass="geheim"

In [ ]:
# TG
tgconn = tg.TigerGraphConnection(host=tghost, username=tguser, password=tgpass, gsqlVersion="3.5.0")

Now, let's start with DDL (vertices and edges)

In [ ]:
!wget https://raw.githubusercontent.com/xpertmind/RecipeNLG/main/setup.gsql -O setup.gsql
with open("./setup.gsql") as f:
    contents = f.read()
print(tgconn.gsql(contents, options=[]))
# now connect to graph 
tgconn.graphname = "RecipeNLG" 
secret = tgconn.createSecret("Workshop_IPYNB")
print("Secret -> "+secret)
token = tgconn.getToken(secret)

--2022-05-06 04:38:59--  https://raw.githubusercontent.com/xpertmind/RecipeNLG/main/setup.gsql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4435 (4.3K) [text/plain]
Saving to: ‘setup.gsql’

setup.gsql          100%[===================>]   4.33K  --.-KB/s    in 0s      

2022-05-06 04:39:00 (52.1 MB/s) - ‘setup.gsql’ saved [4435/4435]

The graph RecipeNLG is created.
Successfully created schema change jobs: [create_ddl_recipe].

Current graph version 0
Trying to add vertex recipes.
Trying to add vertex directions.
Trying to add vertex ingredients.
Trying to add vertex keywords.
Trying to add edge recipe_keyword.
Trying to add edge recipe_ingredient.
Trying to add edge ingredient_recipe.
Trying to add edge howto_cook.
Trying to add edge rev_howto_cook.
Kick

Start data upload

In [ ]:
counter = 0
filter_out_chars = ['\"', '\[', '\]']
pattern = '[' +  ''.join(filter_out_chars) +  ']'

try:
    f = open("./dataset.csv.zip")
except FileNotFoundError:
    print("Datafile not found - downloading!")
    !wget "https://my.graph.place/dl/dataset.csv.zip"

with pd.read_csv("./dataset.csv.zip", compression='zip', delimiter=',', keep_default_na=False, na_values='\\N', chunksize=chunk_size) as df:

      for chunk in df:

        counter += 1
        recipe = []
        recipe_keyword = []
        recipe_ingredient = []
        howto_cook = []
        step = 0

        for row in chunk.itertuples(index=False):
            recipe.append((row[0]+1, {"title": row[1], "link": row[4], "source": row[5]}))
            
            # do some data cleaning based on pattern and filter_out_chars (problem with Pandas recognising list as string)
            item_list = list(row[2].split(","))
            for item in item_list:
                item = re.sub(pattern, '', item).strip()
                recipe_ingredient.append((row[0]+1, item))

            item_list = list(row[6].split(","))
            for item in item_list:
                # do some data cleaning based on pattern and filter_out_chars (problem with Pandas recognising list as string)
                item = re.sub(pattern, '', item).strip()
                recipe_keyword.append((row[0]+1, item))
            
            item_list = list(row[3].split(","))
            for item in item_list:
                step += 1
                item = re.sub(pattern, '', item).strip()
                howto_cook.append((row[0]+1, item, {"step": step}))
            step = 0
        # Upsert chunk_size of records
        tgconn.upsertVertices("recipes", recipe)
        tgconn.upsertEdges("recipes", "recipe_keyword", "keywords", recipe_keyword)
        tgconn.upsertEdges("recipes", "recipe_ingredient", "ingredients", recipe_ingredient)
        tgconn.upsertEdges("recipes", "howto_cook", "directions", howto_cook)
        
        print("------ Chunk " + str(counter))

Datafile not found - downloading!
--2022-05-06 04:39:57--  https://my.graph.place/dl/dataset.csv.zip
Resolving my.graph.place (my.graph.place)... 5.9.78.163
Connecting to my.graph.place (my.graph.place)|5.9.78.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4234329 (4.0M) [application/zip]
Saving to: ‘dataset.csv.zip’

dataset.csv.zip     100%[===================>]   4.04M  3.48MB/s    in 1.2s    

2022-05-06 04:39:58 (3.48 MB/s) - ‘dataset.csv.zip’ saved [4234329/4234329]

------ Chunk 1
------ Chunk 2
------ Chunk 3
------ Chunk 4
------ Chunk 5


Done